In [1]:
#Homework 7
#Mackenzie James
#Due: 4/3/20

#collaborated with: Jimmy Lilly, Madison Walder, Ryan Webster, and Sean Cunningham

In [2]:
# import modules
import numpy as np
import astropy.units as u
import astropy.constants as const

# import plotting modules
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
# import Latex module so we can display the results with symbols
from IPython.display import Latex

# my modules
from ReadFile import Read
from CenterOfMass import CenterOfMass
from GalaxyMass import ComponentMass

In [11]:
### Part 2: Defining All the Acceleration Terms ###


#creates a class that has a series of fucntions that will determine
#the acceleration M33 feels from M31, intergrate current position and 
#velocity forwards in time
class M33AnalyticOrbit:
    
    #initialize the class
    def __init__(self,filename):
    #inputs:
        #filename: takes the file that will will store the integrated orbit
        
        #Setting Gravitational Constant
        self.G = const.G.to(u.kpc**3/u.Msun/u.Gyr**2).value
        
        #store output file name
        self.filename = filename
        
        ##relative COM position and velocity of M33 to M31 (from HW 4)
        #M33 data
        M33COM = CenterOfMass("M33_000.txt", 2)
        M33_COMP = M33COM.COM_P(0.1,4)
        M33_COMV = M33COM.COM_V(M33_COMP[0], M33_COMP[1], M33_COMP[2])
        #M31 data
        M31COM = CenterOfMass("M31_000.txt", 2)
        M31_COMP = M31COM.COM_P(0.1,2)
        M31_COMV = M31COM.COM_V(M31_COMP[0], M31_COMP[1], M31_COMP[2])
        #difference between M31 and M33, setting initial vector
        self.r0 = (M31_COMP - M33_COMP).value
        self.v0 = (M31_COMV - M33_COMV).value
        
        #scale length and masses for each component in M31
        self.rdisk = 5.0 #in kpc
        self.Mdisk = (1e12*ComponentMass("M31_000.txt",2)).value #Msun
        self.rbulge = 1 #kpc
        self.Mbulge = (1e12*ComponentMass("M31_000.txt",2)).value#Msun
        self.rhalo = 60.2 #a for M31 from HW
        self.Mhalo = (1e12*ComponentMass("M31_000.txt",2)).value
        
    
    def HernquistAccel(self,M,ra,r):
    #inputs:
        #M - total halo or bulge mass
        #ra - corresponding scale length
        #r0 - relative initial postion vector 
    #returns: 
        #the acceleration vector from a Hernquist potential
        
        #magnitude of positon vector
        rmag = np.sqrt((r[0]**2)+(r[1]**2)+(r[2]**2))
        
        #acceleration vector
        #prefactor term (all of the constants)
        prefactor = -((self.G*M)/(rmag*((ra+rmag)**2)))
        #acceleration of this factor
        Hern = [x*prefactor for x in r]
        #Hern = prefactor*r
        
        #print(Hern)
        #print(len(Hern))
        return Hern
    
    
    def MiyamotoNagaiAccel(self,M,rd,r):
    #inputs:
        #M - Disk Mass
        #rd - self.rdisk value from __init__ function
        #r0 - relative initial postion vector
    #returns:
        #the acceleration vector for disk particles using MiyamotoNagai potential
        
        
        #magnitude of positon vector
        #rmag = np.sqrt((r[0]**2)+(r[1]**2)+(r[2]**2))
        
        #variables given in assignment above equation (3)
        R = np.sqrt((r[0]**2)+(r[1]**2))
        zd = self.rdisk/5.0
        B = rd + (np.sqrt((r[2]**2)+(zd**2)))
        
        #the components for x and y are the same, so putting them in array as constant C
        C = -((self.G*M)/((R**2+B**2)**1.5))
        
        #now it will be that constant multiplied by an array with 1 for x and y
        # and adding the extra terms for z
        zterms = B/(np.sqrt(r[2]**2+zd**2))
        arrayterms = np.array([1,1,zterms])
        
        #using the prefactor method again for this acceleration term
        prefactor = C*arrayterms
        #now calculate the components
        MiyaNag = np.multiply(np.array(r),prefactor)
        #print(MiyaNag)
        #print(len(MiyaNag))
        return MiyaNag
    
    
    #function that sums all acceleration vectors from each galaxy component
    def M31Accel(self,r):
    #return:
        #The 3D vector of the total acceleration
        
        AcclHalo = self.HernquistAccel(self.Mhalo,self.rhalo,r)
        AcclBulge = self.HernquistAccel(self.Mbulge,self.rbulge,r)
        AcclDisk = self.MiyamotoNagaiAccel(self.Mdisk,self.rdisk,r)
        
        HernTerms = np.add(AcclHalo,AcclBulge) 
        Acc = np.add(HernTerms,AcclDisk)
        print(Acc)
        return Acc
    
### Part 3: Build an Integrator ###
    #function that is using the "Leap Frog" integration for the 
    # orbit of M33
    def LeapFrog(self,dt,r,v):
    #inputs:
        #dt - time interval for integration
        #r - starting postion vector for M33 COM position (relative to M31)
        #v - starting velocity vector for M33 (relative to M31)
    #return:
        #the new postion and velocity vectors

        #predicting position at the next halfstep
        rhalf = r + v*(dt/2)
        
        #predict final velocity at next timestep using the acceleration at halfstep
        vnew = v + self.M31Accel(rhalf)*dt
        
        #predict final position using average of current and final velocity
        rnew = rhalf + vnew*(dt/2)
        
        return rnew, vnew
    
    #function to loop over the integrator to solve the equations of motion
    def OrbitIntegrator(self,t0,dt,tmax):
    #inputs:
        #t0 - starting time
        #dt- a time interval
        #tmax - final time
    #returns
        #future orbit of M33
        
        #initialze time t to the starting time
        t = t0
        
        #initialize an empty array from given parameters (in template)
        orbit = np.zeros((int((tmax/dt)+2),7))
        
        #initialize first row of the orbit
        orbit[0] = t0, *tuple(self.r0), *tuple(self.v0)
        
        #initialize a counter for the orbit
        i = 1
        
        #start integration
        while (t <= tmax):
            t = t + dt #advance the time by one timestep
            
            #store new time in first column of ith row
            orbit[i,0] = t
            
            #advance pos. and vel. by using LeapFrog
            #LeapFrog has two returns, so like position is LeapFrog[0] and veloctiy is LeapFrog [1]
            rnew, vnew = self.LeapFrog(dt,orbit[i-1,1:4],orbit[i-1,4:7])
            
            #store new postion vector
            orbit[i,1:4] = rnew
            #store new velocity vector
            orbit[i,4:7] = vnew
            
            #update counter i """
            i = i + 1
            
        #write data to a file
        np.savetxt(self.filename, orbit, fmt= "%11.3f"*7, comments='#', 
                   header="{:>10s}{:>11s}{:>11s}{:>11s}{:>11s}{:>11s}{:>11s}"\
                   .format('t', 'x', 'y', 'z', 'vx', 'vy', 'vz'))
            
            
        

In [12]:
file = "M33RelativeOrbit.txt"
orbit = M33AnalyticOrbit(file)
orbit.OrbitIntegrator(0.0,0.5,10.0)

[-3.02226909e+13 -2.18748725e+13 -3.04157380e+13]
[7.02140061e-09 5.08201746e-09 7.06624971e-09]
[1.75535015e-09 1.27050436e-09 1.76656243e-09]
[7.80155623e-10 5.64668606e-10 7.85138857e-10]
[4.38837538e-10 3.17626091e-10 4.41640607e-10]
[2.80856024e-10 2.03280698e-10 2.82649989e-10]
[1.95038906e-10 1.41167152e-10 1.96284714e-10]
[1.43293890e-10 1.03714642e-10 1.44209178e-10]
[1.09709384e-10 7.94065228e-11 1.10410152e-10]
[8.66839581e-11 6.27409563e-11 8.72376508e-11]
[7.02140061e-11 5.08201746e-11 7.06624971e-11]
[5.80281042e-11 4.20001443e-11 5.83987580e-11]
[4.87597264e-11 3.52917879e-11 4.90711786e-11]
[4.15467492e-11 3.00711092e-11 4.18121285e-11]
[3.58234725e-11 2.59286605e-11 3.60522945e-11]
[3.12062249e-11 2.25867443e-11 3.14055543e-11]
[2.74273461e-11 1.98516307e-11 2.76025379e-11]
[2.42955038e-11 1.75848355e-11 2.44506910e-11]
[2.16709895e-11 1.56852391e-11 2.18094127e-11]
[1.94498632e-11 1.40776107e-11 1.95740989e-11]
[1.75535015e-11 1.27050436e-11 1.76656243e-11]
